# Malleability of ECDSA Signatures

https://www.derpturkey.com/inherent-malleability-of-ecdsa-signatures

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/masashi_mac_ssd/Developer/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/masashi_mac_ssd/Developer/bitcoin-tx-tutorial"

### Setup bitcoin core test framework
Start up regtest mode, delete any regtest network history so we are starting from scratch. 

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-12-02T19:02:57.598782Z TestFramework (INFO): PRNG seed is: 809076757654615059
2025-12-02T19:02:57.599612Z TestFramework (INFO): Initializing test directory /var/folders/yk/mx8jn78j6xq0w5l4z9d8hbvw0000gn/T/bitcoin_func_test_piiy6hm4


### Create a P2PKH UTXO

In order to create a transaction spending from a P2PKH UTXO, we'll first need to create the UTXO that is locked with a p2pkh script. To do that, we'll create a P2PKH address from a private key, and fund it using the bitcoind wallet created in the setup step.

#### Create a p2pkh address 
For more on this step, review the 'Addresses' notebook.

In [3]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2pkh_addr = pk_to_p2pkh(sender_pubkey, network = "regtest")
print("sender's pubkey: " + sender_pubkey.hex())
print("sender's p2pkh address: " + sender_p2pkh_addr)

sender's pubkey: 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa
sender's p2pkh address: n4XmX91N5FfccY678vaG1ELNtXh6skVES7


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [12]:
txid_to_spend = node.sendtoaddress(sender_p2pkh_addr, 2.001)
print(txid_to_spend)

67998b2d9b607792fb4226c386e1e278d7b3464bf005f68452711f7cc4fbe9c4


We can view the transaction using the bitcoin-cli commands `getrawtransaction` and `decoderawtransaction` as follows:

In [14]:
raw_tx = node.getrawtransaction(txid_to_spend)
decoded = node.decoderawtransaction(raw_tx)
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "67998b2d9b607792fb4226c386e1e278d7b3464bf005f68452711f7cc4fbe9c4",
  "hash": "636ed430b41fbaaee4cd47023503a0c7c729360c2244e1b4c238dbb3ab1a93c4",
  "version": 2,
  "size": 228,
  "vsize": 147,
  "weight": 585,
  "locktime": 102,
  "vin": [
    {
      "txid": "cbd94436d44c8b18644951a53accb4219082e1311dfaadd26c66e4f703d2ed0a",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "30440220311299d13e2224af15ee1c54579b2f5991fea0a7be1b17928cb9f00900a7d63c02205cb88cc3149499271418d08366d181345e3fa0db18d4707833fe7d24d8b3608501",
        "03bbcf80b46a9053bb9066d4846552395da7538a8b7b64648f20d7f13de3ee3a36"
      ],
      "sequence": 4294967293
    }
  ],
  "vout": [
    {
      "value": "2.00100000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 fc7250a211deddc70ee5a2738de5f07817351cef OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(n4XmX91N5FfccY678vaG1ELNtXh6skVES7)#n2xufj4n",
        "hex": 

#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [15]:
if decoded["vout"][0]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 0
elif decoded["vout"][1]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

index to spend from: 0


#### Mine a block so that the funding tx gets confirmed

In [16]:
node.generatetoaddress(1, address, called_by_framework=True);

## Spending a p2pkh UTXO

Now that we have some funds locked up in a p2pkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

### Decoding a base58 address

The first thing we need to do is decode the address. This lets us:  
1 - validate the checksum to know the address was transmitted without error  
2 - make sure we are sending btc on the correct network (testnet/mainnet)  
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the '[Addresses](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/addresses.ipynb)' chapter.

In [17]:
receiver_address = 'mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE'
receiver_address_decoded = decode_base58(receiver_address)
# TODO: create a function in the address chapter to validate and parse addresses and use here

prefix = receiver_address_decoded[0]  
pubkey_hash = receiver_address_decoded[1:-4] 
checksum = receiver_address_decoded[-4:]
print(hex(prefix))
print(pubkey_hash.hex())
print(checksum.hex())

0x6f
3bc28d6d92d9073fb5e3adf481795eaf446bceed
ee2161b7


The first byte , in our case `6f`, tells us that this address corresponds to a p2pkh output for testnet. For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [18]:
receiver_spk = bytes.fromhex("76a914") + pubkey_hash + bytes.fromhex("88ac")

### Create an unsigned p2pkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [19]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the input's scriptSig, the field where the signature goes, is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [20]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  0200000001c4e9fbc47c1f715284f605f04b46b3d778e2e186c32642fb9277609b2d8b99670000000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


Before we can sign this transaction there is one final step we need to do. We need to replace the empty scriptSig with the scriptPubkey of the input we are signing over. If we had multiple inputs, we would need to do this step for each input. We will cover signing transactions with multiple inputs in a later chapter.

Since we are spending from a p2pkh utxo, we will create the scriptPubkey in the same way as we did for the outputs, but using the sender's pubkey:

In [21]:
pk_hash = hash160(sender_pubkey)
input_spk = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk # replace the empty scriptSig with the input scriptPubkey
    + sequence
)

# tx hex to sign
tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [22]:
# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# save the original for debugging purposes
original_signature = signature 

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  0200000001c4e9fbc47c1f715284f605f04b46b3d778e2e186c32642fb9277609b2d8b9967000000006b483045022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb247902205632d6f8464f80a7e628416a96f502c339d80cb1e3a3f176a79312941872d1c70121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [ ]:
# Do not brodcast, try changing it first 
# node.sendrawtransaction(signed_tx.hex())
# node.testmempoolaccept(rawtxs=[signed_tx.hex()])


## Intercept transaction and change its txid

Assume we received a raw transaction via p2p from our victim. Decode, extract the signature and change it.

In [23]:
intercepted_tx = signed_tx.hex()
decoded = node.decoderawtransaction(intercepted_tx)

# save original txid to compare
original_txid = decoded["txid"]

print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "6c7ec6ff1cf5deaf05654451a5ca5cb8ebce1c4f61ff7acc3803c6ed12550522",
  "hash": "6c7ec6ff1cf5deaf05654451a5ca5cb8ebce1c4f61ff7acc3803c6ed12550522",
  "version": 2,
  "size": 226,
  "vsize": 226,
  "weight": 904,
  "locktime": 0,
  "vin": [
    {
      "txid": "67998b2d9b607792fb4226c386e1e278d7b3464bf005f68452711f7cc4fbe9c4",
      "vout": 0,
      "scriptSig": {
        "asm": "3045022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb247902205632d6f8464f80a7e628416a96f502c339d80cb1e3a3f176a79312941872d1c7[ALL] 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa",
        "hex": "483045022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb247902205632d6f8464f80a7e628416a96f502c339d80cb1e3a3f176a79312941872d1c70121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP

In [24]:
# Extract signature and public key from scriptSig
# it is in Distinguished Encoding Rules (DER) format
scriptSig = decoded["vin"][0]["scriptSig"]["hex"]

signature_bytes = bytes.fromhex(scriptSig[2:len(scriptSig)-70])
pubkey_bytes = bytes.fromhex(scriptSig[-66:])

# debug: compare the extracted and original signature
print(signature_bytes.hex() == original_signature.hex())

True


In [25]:
# Decode the DER-encoded signature to obtain 'r' and 's' values
n = ecdsa.SECP256k1.order
r, s = ecdsa.util.sigdecode_der(signature_bytes, n)

print("n:", n)
print("r:", r)
print("s:", s)

n: 115792089237316195423570985008687907852837564279074904382605163141518161494337
r: 92772815484271008876174110652187415798639256349138393999971080872545641768057
s: 38988730998589381124389413919492155348560200847238047990746472438986783773127


In [26]:
# flip the s parameter, calculate new signature, replace it in the decoded transaction
# sigencode_der will allow large s, but sigencode_der_canonize will flip it back

new_s = n - s
new_signature = ecdsa.util.sigencode_der(r, new_s, n)

# in actual wallets the canonize version is used to produce a proper (smaller s) signature
# new_signature = ecdsa.util.sigencode_der_canonize(r, new_s, signing_key.curve.order)

print("Old signature:", original_signature.hex())
print("New signature:", new_signature.hex())

Old signature: 3045022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb247902205632d6f8464f80a7e628416a96f502c339d80cb1e3a3f176a79312941872d1c7
New signature: 3046022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb2479022100a9cd2907b9b07f5819d7be95690afd3b80d6d034cba4aec5183f4bf8b7c36f7a


In [29]:
# Append SIGHASH_ALL to the signature
new_signature = new_signature + bytes.fromhex("01")

# Signature
new_sig_script_signed = (
    pushbytes(new_signature)
    + pushbytes(pubkey_bytes)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(new_sig_script_signed)
    + new_sig_script_signed
    + sequence
)

# the final signed transaction
new_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

new_tx_decoded = node.decoderawtransaction(new_tx.hex())
print(json.dumps(new_tx_decoded, indent=2, default=str))

{
  "txid": "17d84487480031e3b8668150b550136809a59e82d9b358a27c65d708bd6c8b34",
  "hash": "17d84487480031e3b8668150b550136809a59e82d9b358a27c65d708bd6c8b34",
  "version": 2,
  "size": 228,
  "vsize": 228,
  "weight": 912,
  "locktime": 0,
  "vin": [
    {
      "txid": "67998b2d9b607792fb4226c386e1e278d7b3464bf005f68452711f7cc4fbe9c4",
      "vout": 0,
      "scriptSig": {
        "asm": "3046022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb2479022100a9cd2907b9b07f5819d7be95690afd3b80d6d034cba4aec5183f4bf8b7c36f7a0101 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa",
        "hex": "4a3046022100cd1b8d821bc4de36f8adf7014c223779f2df8a656e8a0f01faffafa4febb2479022100a9cd2907b9b07f5819d7be95690afd3b80d6d034cba4aec5183f4bf8b7c36f7a010121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_D

## Try broadcasting the amended transaction

This will work if the BIP 146 check is disabled in reference client:
https://github.com/bitcoin/bitcoin/blob/35fe0393f216aa6020fc929272118eade5628636/src/script/interpreter.cpp#L185

I commented out return `set_error(serror, SCRIPT_ERR_SIG_HIGH_S);` line in `IsLowDERSignature`

Otherwise, this should raise JSONRPCException:
non-mandatory-script-verify-flag (Non-canonical signature: S value is unnecessarily high) (-26)

In [30]:
print("Old txid: " + original_txid)
node.sendrawtransaction(new_tx.hex())

Old txid: 6c7ec6ff1cf5deaf05654451a5ca5cb8ebce1c4f61ff7acc3803c6ed12550522


JSONRPCException: mempool-script-verify-flag-failed (Non-canonical DER signature), input 0 of 17d84487480031e3b8668150b550136809a59e82d9b358a27c65d708bd6c8b34 (wtxid 17d84487480031e3b8668150b550136809a59e82d9b358a27c65d708bd6c8b34), spending 67998b2d9b607792fb4226c386e1e278d7b3464bf005f68452711f7cc4fbe9c4:0 (-26)

In [32]:
# stop bitcoin core
test.shutdown()

2023-09-18T01:24:14.129000Z TestFramework (INFO): Stopping nodes
2023-09-18T01:24:14.231000Z TestFramework (INFO): Cleaning up C:\Users\Vlad\AppData\Local\Temp\bitcoin_func_test_467l8n4w on exit
2023-09-18T01:24:14.231000Z TestFramework (INFO): Tests successful
